In [105]:
# Importamos librerias
import requests
import zipfile
import pandas as pd
import io
import json
from pathlib import Path

<h3 style="color:yellow">Consulta a la API de AEMET</h3>

In [ ]:
# API Key
AEMET_API_KEY = "---"

# Estaciones Madrid
ESTACION_RETIRO = "3195"
ESTACION_CUATRO_VIENTOS = "3196"
ESTACION_BARAJAS = "3129"

# Fechas fi: fecha inicio / ff: fecha fin. 
# En 2 partes, ya que no permiten descargar datos de más de 6 meses
fi_1 = "2023-12-31T23:00:00UTC" # 01/01/24 00:00:00 en madrid
ff_1 = "2024-06-30T21:59:59UTC" # 30/06/24 23:59:59 en madrid
fi_2 = "2024-06-30T22:00:00UTC" # 01/07/24 00:00:00 en madrid
ff_2 = "2024-12-31T22:59:59UTC" # 31/12/24 23:59:59 en madrid

# se necesita una URL con API Key para obtener otra URL desde donde obtener los datos concretos
url = (
    f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/"
    f"datos/fechaini/{fi_2}/fechafin/{ff_2}/estacion/{ESTACION_RETIRO}"
)

In [167]:
response = requests.get(url, params={'api_key': AEMET_API_KEY})

In [168]:
print(response.status_code)
response.json()

200


{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/aa6992e0',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

In [169]:
data_url = response.json()['datos']
meta_url = response.json()['metadatos']

In [170]:
data_response = requests.get(data_url)
metadata_response = requests.get(meta_url)

<h3 style="color:yellow">Datos de Ene-Jun y Jul-Dic</h3>
<p style="color:yellow">Finalmente sólo cogeré los datos de Retiro</p>

In [164]:
df_1 = pd.read_json(io.BytesIO(data_response.content), orient='records')

In [171]:
df_1.head(2)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaracha,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-01-01,3195,"MADRID, RETIRO",MADRID,667,"6,6","0,0","3,8",08:50,"9,4",...,15:50,"945,1",23,"940,8",03,85.0,99.0,Varias,62.0,15:30
1,2024-01-02,3195,"MADRID, RETIRO",MADRID,667,"5,5","2,0","2,8",05:50,"8,2",...,02:10,"945,2",11,"941,7",24,92.0,99.0,Varias,83.0,13:20


In [172]:
df_2 = pd.read_json(io.BytesIO(data_response.content), orient='records')

In [173]:
df_2.head(2)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaracha,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-07-01,3195,"MADRID, RETIRO",MADRID,667,"24,1","0,0","18,2",05:10,"30,0",...,21:50,"943,7",23,"938,5",03,40.0,69.0,04:30,24.0,17:10
1,2024-07-02,3195,"MADRID, RETIRO",MADRID,667,"23,8","0,0","16,0",05:30,"31,6",...,19:20,"944,1",07,"939,0",18,39.0,69.0,05:30,25.0,14:50


In [175]:
path_1 = Path("../docs/aemet_datos_retiro_2024_1.json")
path_2 = Path("../docs/aemet_datos_retiro_2024_2.json")
path_meta = Path("../docs/aemet_metadatos_retiro_2024.json")

df_1.to_json(path_1, orient='records', force_ascii=False, indent=2)
df_2.to_json(path_2, orient='records', force_ascii=False, indent=2)
metadata_df.to_json(path_meta, orient='records', force_ascii=False, indent=2)

print(f'Datos guardados en {path_1.resolve()} y {path_2.resolve()}.')
print(f'Metadatos guardados en {path_meta.resolve()}.')

Datos guardados en C:\Users\xabi\Documents\parking_tickets_ser_madrid\docs\aemet_datos_retiro_2024_1.json y C:\Users\xabi\Documents\parking_tickets_ser_madrid\docs\aemet_datos_retiro_2024_2.json.
Metaatos guardados en C:\Users\xabi\Documents\parking_tickets_ser_madrid\docs\aemet_metadatos_retiro_2024.json.


## Metadatos

In [130]:
metadata_df = pd.read_json(metadata_response.text, orient='records')

C:\Users\xabi\AppData\Local\Temp\ipykernel_12600\2848166145.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  metadata_df = pd.read_json(metadata_response.text, orient='records')


In [106]:
path = Path("../docs/aemet_metadatos_retiro_2024.json")
metadata_df.to_json(path, orient='records', force_ascii=False, indent=2)

print(f'Metadatos guardados en {path.resolve()}.')

Metadatos guardados en C:\Users\xabi\Documents\parking_tickets_ser_madrid\docs\aemet_metadatos_retiro_2024.json.


In [180]:
metadata_df['campos'][6]

{'id': 'prec',
 'descripcion': 'Precipitación diaria de 07 a 07',
 'tipo_datos': 'float',
 'unidad': 'mm (Ip = inferior a 0,1 mm) (Acum = Precipitación acumulada)',
 'requerido': False}

<div style="text-align:center">
<h3 style="color:yellow;;">⚠⚠⚠⚠⚠⚠⚠⚠⚠⚠⚠<br/> Este dataset sólo ofrece datos de precipitaciones diarias, no por horas</h3>
</div>

### Creando un único dataset con los datos diarios de 2024 para Retiro

In [182]:
df = pd.concat([df_1, df_2],
    axis=0,
    ignore_index=True
)

df = df.sort_values("fecha").reset_index(drop=True)

In [187]:
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaracha,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-01-01,3195,"MADRID, RETIRO",MADRID,667,"6,6","0,0","3,8",08:50,"9,4",...,15:50,"945,1",23,"940,8",03,85.0,99.0,Varias,62.0,15:30
1,2024-01-02,3195,"MADRID, RETIRO",MADRID,667,"5,5","2,0","2,8",05:50,"8,2",...,02:10,"945,2",11,"941,7",24,92.0,99.0,Varias,83.0,13:20
2,2024-01-03,3195,"MADRID, RETIRO",MADRID,667,"10,1","2,7","8,1",00:00,"12,1",...,16:00,"942,7",10,"940,1",18,99.0,99.0,Varias,91.0,22:00
3,2024-01-04,3195,"MADRID, RETIRO",MADRID,667,"9,4","9,7","8,2",00:40,"10,6",...,19:50,"941,9",00,"931,8",24,99.0,99.0,Varias,96.0,11:50
4,2024-01-05,3195,"MADRID, RETIRO",MADRID,667,"6,6","0,1","4,6",06:20,"8,6",...,22:00,"936,3",24,"929,8",05,71.0,99.0,Varias,53.0,15:40


In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   fecha        366 non-null    object 
 1   indicativo   366 non-null    int64  
 2   nombre       366 non-null    object 
 3   provincia    366 non-null    object 
 4   altitud      366 non-null    int64  
 5   tmed         358 non-null    object 
 6   prec         366 non-null    object 
 7   tmin         358 non-null    object 
 8   horatmin     358 non-null    object 
 9   tmax         358 non-null    object 
 10  horatmax     358 non-null    object 
 11  dir          361 non-null    float64
 12  velmedia     363 non-null    object 
 13  racha        361 non-null    object 
 14  horaracha    361 non-null    object 
 15  presMax      348 non-null    object 
 16  horaPresMax  348 non-null    object 
 17  presMin      347 non-null    object 
 18  horaPresMin  347 non-null    object 
 19  hrMedia 

### Guardamos los datos en CSV

In [189]:
df.to_csv('../data/aemet_datos_retiro_2024.csv', index=False, encoding='utf-8')